# Pasta Mouse

## Index
1. [Instantiate model class](#Instantiate-model-class)
2. [Define clock metadata](#Define-clock-metadata)
3. [Download clock dependencies](#Download-clock-dependencies)
4. [Load features](#Load-features)
5. [Map to mouse orthologs](#Map-to-mouse-orthologs)
6. [Load weights into base model](#Load-weights-into-base-model)
7. [Load reference values](#Load-reference-values)
8. [Load preprocess and postprocess objects](#Load-preprocess-and-postprocess-objects)
9. [Check all clock parameters](#Check-all-clock-parameters)
10. [Basic test](#Basic-test)
11. [Save torch model](#Save-torch-model)
12. [Clear directory](#Clear-directory)


Let's first import some packages:

In [1]:
import os
import inspect
import shutil
import json
import subprocess

import torch
import pandas as pd
import pyaging as pya

## Instantiate model class

In [2]:
def print_entire_class(cls):
    source = inspect.getsource(cls)
    print(source)

print_entire_class(pya.models.PastaMouse)

class PastaMouse(Pasta):
    def __init__(self):
        super().__init__()
        self.base_model_features = None
        self.mouse_feature_indices = None
        self.full_reference_values = None

    def set_mouse_features(self, full_features, full_reference_values=None, mouse_prefix="ENSMUSG"):
        """
        Configure the mouse-only interface while keeping the full feature space for the base model.
        """
        self.base_model_features = list(full_features)
        self.full_reference_values = full_reference_values

        self.mouse_feature_indices = [
            i
            for i, feature in enumerate(self.base_model_features)
            if isinstance(feature, str) and feature.startswith(mouse_prefix)
        ]

        if len(self.mouse_feature_indices) == 0:
            raise ValueError("No mouse features were identified when configuring PastaMouse.")

        self.features = [self.base_model_features[i] for i in self.mouse_feature_indices]

        if self.

In [3]:
model = pya.models.PastaMouse()

## Define clock metadata

In [4]:
model.metadata["clock_name"] = 'pastamouse'
model.metadata["data_type"] = 'transcriptomics'
model.metadata["species"] = 'Mus musculus'
model.metadata["year"] = 2025
model.metadata["approved_by_author"] = '✅'
model.metadata["citation"] = 'Salignon, Jerome, et al. "Pasta, an age-shift transcriptomic clock, maps the chemical and genetic determinants of aging and rejuvenation." bioRxiv (2025): 2025-06.'
model.metadata["doi"] = "https://doi.org/10.1101/2025.06.04.657785"
model.metadata["research_only"] = None
model.metadata["notes"] = 'Rank-normalized Pasta clock using mouse one-to-one ortholog genes when available.'

## Download clock dependencies

#### Download coefficient file

In [5]:
coeff_url = "https://raw.githubusercontent.com/bio-learn/biolearn/master/biolearn/data/Pasta.csv"
os.system(f"curl -L {coeff_url} -o Pasta.csv")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  322k  100  322k    0     0  1477k      0 --:--:-- --:--:-- --:--:-- 1478k


0

#### Download ortholog mapping

In [6]:
ortholog_url = "https://raw.githubusercontent.com/jsalignon/pasta/main/data/v_human_mouse_one2one.rda"
os.system(f"curl -L {ortholog_url} -o v_human_mouse_one2one.rda")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 24796  100 24796    0     0   129k      0 --:--:-- --:--:-- --:--:--  129k


0

## Load features

#### From CSV file

In [7]:
coeffs = pd.read_csv('Pasta.csv')
coeffs['feature'] = coeffs['GeneID']
coeffs['coefficient'] = coeffs['CoefficientTraining']

model.features = coeffs['feature'].tolist()

## Map to mouse orthologs

In [8]:
r_cmd = (
    "load('v_human_mouse_one2one.rda'); "
    "df <- data.frame(mouse=names(v_human_mouse_one2one), human=as.character(v_human_mouse_one2one)); "
    "write.csv(df, 'v_human_mouse_one2one.csv', row.names=FALSE)"
)
os.system(f"Rscript -e \"{r_cmd}\"")

ortholog_df = pd.read_csv('v_human_mouse_one2one.csv')
human_to_mouse = dict(zip(ortholog_df['human'], ortholog_df['mouse']))

mapped_features = [human_to_mouse.get(gene, gene) for gene in model.features]
mapped_count = sum(gene in human_to_mouse for gene in model.features)
print(f"Mapped {mapped_count} of {len(model.features)} features to mouse orthologs.")
model.features = mapped_features

Mapped 1600 of 8113 features to mouse orthologs.


In [9]:
import numpy as np
len(np.intersect1d(list(human_to_mouse.keys()), list(coeffs['feature'])))

1600

In [10]:
len(np.unique(list(coeffs['feature'])))

8113

## Load weights into base model

#### From CSV file

In [11]:
weights = torch.tensor(coeffs['coefficient'].tolist()).unsqueeze(0)
intercept = torch.tensor([0.0])

#### Linear model

In [12]:
base_model = pya.models.LinearModel(input_dim=len(model.features))

base_model.linear.weight.data = weights.float()
base_model.linear.bias.data = intercept.float()

model.base_model = base_model

## Load reference values

In [13]:
full_features = list(model.features)
full_reference_values = [float('nan')] * len(full_features)
model.reference_values = full_reference_values
model.set_mouse_features(full_features, full_reference_values)


## Load preprocess and postprocess objects

In [14]:
model.preprocess_name = "median_fill_and_rank_normalization"
model.preprocess_dependencies = None

In [15]:
model.postprocess_name = "scale_and_shift"
model.postprocess_dependencies = [-4.76348378687217, -0.0502893445253186]

## Check all clock parameters

In [16]:
pya.utils.print_model_details(model)


%==================================== Model Details ====================================%
Model Attributes:

training: True
metadata: {'approved_by_author': '✅',
 'citation': 'Salignon, Jerome, et al. "Pasta, an age-shift transcriptomic '
             'clock, maps the chemical and genetic determinants of aging and '
             'rejuvenation." bioRxiv (2025): 2025-06.',
 'clock_name': 'pastamouse',
 'data_type': 'transcriptomics',
 'doi': 'https://doi.org/10.1101/2025.06.04.657785',
 'notes': 'Rank-normalized Pasta clock using mouse one-to-one ortholog genes '
          'when available.',
 'research_only': None,
 'species': 'Mus musculus',
 'version': None,
 'year': 2025}
reference_values: [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]... [Total elements: 1600]
preprocess_name: 'median_fill_and_rank_normalization'
preprocess_dependencies: None
postprocess_name: 'scale_and_shift'
po

## Basic test

In [17]:
torch.manual_seed(42)
input = torch.randn(10, len(model.features), dtype=float)
model.eval()
model.to(float)
pred = model(input)
pred

tensor([[  4.9918],
        [ -8.4652],
        [ 15.1181],
        [-31.3271],
        [ 27.5393],
        [-10.9938],
        [ -3.4235],
        [-14.1880],
        [-24.5564],
        [ -7.9826]], dtype=torch.float64, grad_fn=<AddBackward0>)

## Save torch model

In [18]:
torch.save(model, f"../weights/{model.metadata['clock_name']}.pt")

## Clear directory
<a id="10"></a>

In [19]:
# Function to remove a folder and all its contents
def remove_folder(path):
    try:
        shutil.rmtree(path)
        print(f"Deleted folder: {path}")
    except Exception as e:
        print(f"Error deleting folder {path}: {e}")

# Get a list of all files and folders in the current directory
all_items = os.listdir('.')

# Loop through the items
for item in all_items:
    # Check if it's a file and does not end with .ipynb
    if os.path.isfile(item) and not item.endswith('.ipynb'):
        os.remove(item)
        print(f"Deleted file: {item}")
    # Check if it's a folder
    elif os.path.isdir(item):
        remove_folder(item)

Deleted file: v_human_mouse_one2one.rda
Deleted file: Pasta.csv
Deleted file: v_human_mouse_one2one.csv
